In [3]:
print("all ok")

all ok


In [5]:
# Ensure project root is on sys.path so local packages like `logger` can be imported
import sys
from pathlib import Path
proj_root = Path.cwd().parent  # notebook is in `notebook/`
if str(proj_root) not in sys.path:
    sys.path.insert(0, str(proj_root))
from logger.custom_logger import CustomLogger

In [6]:
logger=CustomLogger().get_logger("exception_experiment")

In [10]:
import sys
import traceback
import textwrap
class DocumentPortalException(Exception):
    """Custom exception for Document Portal"""
    def __init__(self, error_message, error_details: sys):
        # Capture exc_info and derive file/line + formatted traceback string
        _, _, exc_tb = error_details.exc_info()
        self.file_name = exc_tb.tb_frame.f_code.co_filename
        self.lineno = exc_tb.tb_lineno
        self.error_message = str(error_message)
        self.traceback_str = ''.join(traceback.format_exception(*error_details.exc_info()))

    def __str__(self):
        # Return a concise one-line summary (traceback logged separately)
        return f"Error in [{self.file_name}] at line [{self.lineno}] - {self.error_message}"

    def to_dict(self):
        # Structured data useful for telemetry or JSON serialization
        return {
            'file': self.file_name,
            'line': self.lineno,
            'message': self.error_message,
            'traceback': self.traceback_str,
        }

if __name__ == "__main__":
    try:
        # Simulate an error
        a = 1 / 0
        print(a)
    except Exception as e:
        app_exc = DocumentPortalException(e, sys)
        # Log concise message and include traceback once via logger.exception
        logger.exception(str(app_exc))
        # Re-raise without original context to avoid duplicate traceback display
        raise app_exc from None


2025-12-24 18:51:04,899 ERROR exception_experiment: Error in [/tmp/ipykernel_301660/3432023705.py] at line [30] - division by zero
Traceback (most recent call last):
  File "/tmp/ipykernel_301660/3432023705.py", line 30, in <module>
    a = 1 / 0
        ~~^~~
ZeroDivisionError: division by zero


DocumentPortalException: Error in [/tmp/ipykernel_301660/3432023705.py] at line [30] - division by zero